---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])

        else:
            town = line
            state_town.append([state,town])
    state_college_df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    return state_college_df

In [4]:
def get_recession_start():
    
    df = pd.read_excel('gdplev.xls')
    df.columns = ['Current-Dollar and Real Gross Domestic Product', 'GDP in billions of current dollars', 'del', 'del1', 'Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars', 'del2']
    df.drop(['del', 'del1', 'del2'], axis = 1, inplace = True)
    df = df.iloc[8:]
    df.index -= 8
    df = df.iloc[211:]
    df = df[['Quarter', 'GDP in billions of chained 2009 dollars']]
    df.index -= 211
    dec_quarter = list()

    for i in range(1,65):
    
        if ( df.iloc[i]['GDP in billions of chained 2009 dollars'] > df.loc[i+1]['GDP in billions of chained 2009 dollars'] and df.loc[i+1]['GDP in billions of chained 2009 dollars'] > df.loc[i+2]['GDP in billions of chained 2009 dollars']) and ( df.iloc[i+2]['GDP in billions of chained 2009 dollars'] < df.loc[i+3]['GDP in billions of chained 2009 dollars'] and df.iloc[i+3]['GDP in billions of chained 2009 dollars'] < df.loc[i+4]['GDP in billions of chained 2009 dollars']):
            dec_quarter.append([df.iloc[i]['Quarter'],df.iloc[i+4]['Quarter']])
        
        
    return('2008q3')  

In [5]:
def get_recession_end():

    return "2009q4"

In [6]:
def get_recession_bottom():

    return '2009q2'

In [7]:
def convert_housing_data_to_quarters():

     states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
     temp_df = pd.DataFrame(states,index=[0])
     temp_df = temp_df.T
     df = pd.read_csv('City_Zhvi_AllHomes.csv')

     df.set_index(['State'], inplace = True)
     df = df.merge(temp_df,left_index=True,right_index=True)
     df.rename(columns = {0:'State'},inplace = True)
     df.set_index(['State','RegionName'],inplace = True)
     df.columns[:49]
     df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank', '1996-04', '1996-05',
            '1996-06', '1996-07', '1996-08', '1996-09', '1996-10', '1996-11',
            '1996-12', '1997-01', '1997-02', '1997-03', '1997-04', '1997-05',
            '1997-06', '1997-07', '1997-08', '1997-09', '1997-10', '1997-11',
            '1997-12', '1998-01', '1998-02', '1998-03', '1998-04', '1998-05',
            '1998-06', '1998-07', '1998-08', '1998-09', '1998-10', '1998-11',
            '1998-12', '1999-01', '1999-02', '1999-03', '1999-04', '1999-05',
            '1999-06', '1999-07', '1999-08', '1999-09', '1999-10', '1999-11',
            '1999-12'], axis = 1, inplace = True)

     res = (df.groupby(pd.PeriodIndex(df.columns, freq='Q'), axis=1)
                         .mean()
                         .rename(columns=lambda c: str(c).lower()))
         
     return res

In [54]:
df1 = convert_housing_data_to_quarters()
df1 = df1[['2008q3','2008q4', '2009q1','2009q2']]
df1.reset_index(inplace = True)
df1['Diff'] = df1['2009q2'] - df1['2008q3']
df1.dropna(how='any',inplace = True)

uni_df = get_list_of_university_towns()
uni_df['university'] = True

university_data = get_list_of_university_towns().reset_index(drop=True).merge(df1, how='inner', left_on=['State','RegionName'], right_on=['State','RegionName'])
university_data['isuni'] = True
df2 = df1.merge(university_data,how='outer', left_on=['State','RegionName'],right_on=['State','RegionName'])
df2['isuni'].fillna(value = False,inplace = True)

non_university_data = df2[df2['isuni']==False]
non_university_data = non_university_data[['State', 'RegionName', '2008q3_x', '2008q4_x', '2009q1_x', '2009q2_x','Diff_x']]
non_university_data.columns = df1.columns
university_data.drop(['isuni'],axis = 1,inplace =True)
non_university_data.set_index(['State','RegionName'], inplace = True)
university_data.set_index(['State','RegionName'],inplace = True)

stats.ttest_ind(university_data['Diff'],non_university_data['Diff'],axis = 0)

Ttest_indResult(statistic=2.8540746960114096, pvalue=0.0043252148535112009)

In [53]:
def run_ttest():
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    
    
    return (True,0.0043252148535112009,"university town")